In [ ]:
import pandas as pd
import re
from mlxtend.frequent_patterns import apriori, association_rules
data = pd.read_csv("Team_Sports_Survey_Preprocessed.csv")
# remove the eg. for each items
def remove_parentheses(text):
    if isinstance(text, str):
        return re.sub(r"\s*\(.*?\)", "", text)
    return text

In [ ]:
# Get the rows which preferred days is weekend
# Split the values into a single items in the "Preferred Check-in Days" column
data["Preferred Check-in Days"] = data["Preferred Check-in Days"].str.split(", ")
# Get only rows where Saturday or Sunday is in the dataset
weekend_data = data[data["Preferred Check-in Days"].apply(lambda x: any(day in ["Saturday", "Sunday"] for day in x))]

In [ ]:
# Split the values into a single items
weekend_data["Preferred Spa and Wellness Activities"] = weekend_data["Preferred Spa and Wellness Activities"].apply(remove_parentheses).str.split(", ")
# Convert the items into a transactional data
transactions = weekend_data["Preferred Spa and Wellness Activities"].explode().str.strip().reset_index()
transactions_encoded = transactions.pivot_table(index="index", columns="Preferred Spa and Wellness Activities", aggfunc=lambda x: 1, fill_value=0)

In [ ]:
# Use Apriori algorithm to find frequent association
frequent_items= apriori(transactions_encoded, min_support=0.3, use_colnames=True)
rule = association_rules(frequent_items)
rule[['antecedents','consequents','support', 'confidence', 'lift']].sort_values("confidence", ascending=False)